In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

from lol_win_predictor import (
    load_and_preprocess_data,
    aggregate_players_by_team,
    identify_key_features,
    train_win_rate_model,
    train_player_to_team_features,
    predict_win_rate_from_players,
    evaluate_team_composition,
    print_team_analysis,
    generate_random_player_stats,
    create_team_variants,
    create_optimal_team,
    save_models,
    load_models,
    plot_team_comparison,
    plot_role_impact,
    plot_optimization_progress
)

2025-03-21 21:49:43.598540: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-21 21:49:43.607028: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742618983.616800  276647 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742618983.620268  276647 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 21:49:43.631156: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Define folder paths
models_folder = 'models'
data_folder = 'data'

# Create models folder if it doesn't exist
if not os.path.exists(models_folder):
    os.makedirs(models_folder)
    print(f"Created folder: {models_folder}")

model_file_path = os.path.join(models_folder, 'lol_models.pkl')

if os.path.exists(model_file_path):
    print(f"\nLoading existing models from {model_file_path}...")
    feature_to_win_rate_model, player_to_feature_models, key_features = load_models(models_folder)
    print("Models loaded successfully!")
else:
    print(f"\nNo existing models found at {model_file_path}. Training new models...")
    
    # Load and preprocess data
    df_players, df_teams = load_and_preprocess_data(data_folder)
    
    # Aggregate player statistics by team
    print("\nAggregating player statistics by team...")
    df_aggregated = aggregate_players_by_team(df_players)
    
    # Identify key features for predicting win rate
    print("\nIdentifying key team features for win rate prediction...")
    key_features = identify_key_features(df_teams, n_features=8)
    
    # Train model to predict win rate from team features
    print("\nTraining win rate prediction model using key team features...")
    feature_to_win_rate_model = train_win_rate_model(df_teams, key_features)
    
    # Train models to predict team features from player statistics
    print("\nTraining models to predict key team features from player statistics...")
    player_to_feature_models = train_player_to_team_features(df_aggregated, df_teams, key_features)
    
    # Save the trained models
    save_models(feature_to_win_rate_model, player_to_feature_models, key_features, models_folder)

print(f"\nKey features used for prediction: {key_features}")

Created folder: models

No existing models found at models/lol_models.pkl. Training new models...
Loading data from data/players_stats.csv and data/teams_stats.csv...
Preprocessing data...
Found rosters file at data/rosters.csv. Trying to merge team information...

Aggregating player statistics by team...

Identifying key team features for win rate prediction...
Top features for predicting win rate:
Towers lost: 396.5344
GPM: 332.1408
K:D: 305.6922
GDM: 281.9055
Towers killed: 204.0661
NASH%: 140.4598
Kills / game: 115.0073
DPM: 60.2164

Training win rate prediction model using key team features...

1. Linear Regression Results:
RMSE: 0.0478
R²: 0.8608

2. Random Forest Results:
RMSE: 0.0446
R²: 0.8787

Random Forest Feature Importance:
Towers lost: 0.2232
GDM: 0.2201
K:D: 0.1570
GPM: 0.1515
Towers killed: 0.1306
NASH%: 0.0605
DPM: 0.0291
Kills / game: 0.0281


I0000 00:00:1742618986.323354  276647 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13499 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1742618987.759002  276779 service.cc:148] XLA service 0x7f69f8009ca0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742618987.759028  276779 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 SUPER, Compute Capability 8.9
2025-03-21 21:49:47.792238: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742618987.890671  276779 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1742618988.381231  276779 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step

3. Neural Network Results:
RMSE: 0.0529
R²: 0.8291

Best model: Random Forest with R² = 0.8787

Training models to predict key team features from player statistics...

=== Training models to predict K:D ===
1. Linear Regression - RMSE: 0.0688, R²: 0.9367
2. Random Forest - RMSE: 0.1510, R²: 0.6950
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
3. Neural Network - RMSE: 0.3618, R²: -0.7516
Best model for K:D: Linear Regression with R² = 0.9367

=== Training models to predict GPM ===
1. Linear Regression - RMSE: 22.5157, R²: 0.8969
2. Random Forest - RMSE: 33.4494, R²: 0.7725
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
3. Neural Network - RMSE: 1033.9133, R²: -216.3848
Best model for GPM: Linear Regression with R² = 0.8969

=== Training models to predict GDM ===
1. Linear Regression - RMSE: 35.9586, R²: 0.9304
2. Random Forest - RMSE: 62.6455, R²: 0.7887
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
3. Neural Network - RMSE: 49.5764, R²: 0.8677
Best model for GDM: Lin

In [3]:
strong_team = {
    'TOP': {'KDA': 3.5, 'Avg kills': 2.8, 'Avg deaths': 1.8, 'Avg assists': 5.5, 
            'CSM': 8.5, 'GPM': 400, 'DMG%': 0.24, 'DPM': 480, 'Avg WPM': 0.65,
            'GD@15': 150, 'CSD@15': 12, 'XPD@15': 200},
    'JUNGLE': {'KDA': 4.0, 'Avg kills': 3.0, 'Avg deaths': 2.0, 'Avg assists': 9.0, 
               'CSM': 5.5, 'GPM': 380, 'DMG%': 0.19, 'DPM': 420, 'Avg WPM': 0.9,
               'GD@15': 180, 'CSD@15': 15, 'XPD@15': 220},
    'MID': {'KDA': 4.5, 'Avg kills': 4.2, 'Avg deaths': 1.6, 'Avg assists': 6.5, 
            'CSM': 9.0, 'GPM': 430, 'DMG%': 0.32, 'DPM': 550, 'Avg WPM': 0.6,
            'GD@15': 200, 'CSD@15': 18, 'XPD@15': 250},
    'ADC': {'KDA': 5.0, 'Avg kills': 4.8, 'Avg deaths': 1.3, 'Avg assists': 6.0, 
            'CSM': 9.5, 'GPM': 450, 'DMG%': 0.35, 'DPM': 600, 'Avg WPM': 0.5,
            'GD@15': 250, 'CSD@15': 22, 'XPD@15': 280},
    'SUPPORT': {'KDA': 4.5, 'Avg kills': 1.2, 'Avg deaths': 2.0, 'Avg assists': 14.0, 
                'CSM': 1.2, 'GPM': 280, 'DMG%': 0.12, 'DPM': 250, 'Avg WPM': 1.8,
                'GD@15': 50, 'CSD@15': 2, 'XPD@15': 30}
}

# Create a weaker team
weak_team = {
    'TOP': {'KDA': 2.0, 'Avg kills': 1.5, 'Avg deaths': 3.0, 'Avg assists': 3.5, 
            'CSM': 7.0, 'GPM': 340, 'DMG%': 0.18, 'DPM': 380, 'Avg WPM': 0.4,
            'GD@15': -150, 'CSD@15': -8, 'XPD@15': -120},
    'JUNGLE': {'KDA': 2.2, 'Avg kills': 1.5, 'Avg deaths': 3.2, 'Avg assists': 5.5, 
               'CSM': 4.2, 'GPM': 320, 'DMG%': 0.15, 'DPM': 350, 'Avg WPM': 0.5,
               'GD@15': -80, 'CSD@15': -6, 'XPD@15': -100},
    'MID': {'KDA': 2.5, 'Avg kills': 2.2, 'Avg deaths': 3.0, 'Avg assists': 4.0, 
            'CSM': 7.2, 'GPM': 360, 'DMG%': 0.22, 'DPM': 400, 'Avg WPM': 0.4,
            'GD@15': -100, 'CSD@15': -7, 'XPD@15': -150},
    'ADC': {'KDA': 2.8, 'Avg kills': 2.5, 'Avg deaths': 2.8, 'Avg assists': 3.8, 
            'CSM': 7.8, 'GPM': 370, 'DMG%': 0.24, 'DPM': 420, 'Avg WPM': 0.3,
            'GD@15': -50, 'CSD@15': -4, 'XPD@15': -80},
    'SUPPORT': {'KDA': 2.2, 'Avg kills': 0.6, 'Avg deaths': 3.8, 'Avg assists': 7.5, 
                'CSM': 0.8, 'GPM': 220, 'DMG%': 0.07, 'DPM': 180, 'Avg WPM': 1.1,
                'GD@15': -30, 'CSD@15': -2, 'XPD@15': -50}
}

# Generate a random team
random_team = generate_random_player_stats()

In [4]:
print("\nAnalyzing strong team...")
strong_win_rate, strong_analysis = evaluate_team_composition(
    strong_team, player_to_feature_models, feature_to_win_rate_model, key_features
)
print_team_analysis(strong_analysis)

print("\nAnalyzing weak team...")
weak_win_rate, weak_analysis = evaluate_team_composition(
    weak_team, player_to_feature_models, feature_to_win_rate_model, key_features
)
print_team_analysis(weak_analysis)

print("\nAnalyzing random team...")
random_win_rate, random_analysis = evaluate_team_composition(
    random_team, player_to_feature_models, feature_to_win_rate_model, key_features
)
print_team_analysis(random_analysis)


Analyzing strong team...

----- Team Composition Analysis -----
Predicted Win Rate: 66.58%

Predicted Team Stats:
- K:D: 1.5775
- GPM: 1940.3921
- GDM: 249.7160
- Kills / game: 14.6191
- Towers killed: 9.0807
- Towers lost: 2.7715
- NASH%: 80.7685
- DPM: 2286.5295

Role Impact Analysis:
- TOP: Impact Score = 8.69
- JUNGLE: Impact Score = 7.41
- MID: Impact Score = 11.52
- ADC: Impact Score = 12.65
- SUPPORT: Impact Score = 5.43

Team Strengths:
- K:D: 1.58
- GPM: 1940.39
- GDM: 249.72
- Kills / game: 14.62
- DPM: 2286.53

No significant weaknesses detected.

Recommendations:
- This team composition has good potential for success.
- Maintain consistent performance to achieve the predicted win rate.

Analyzing weak team...

----- Team Composition Analysis -----
Predicted Win Rate: 26.13%

Predicted Team Stats:
- K:D: 0.4378
- GPM: 1628.1525
- GDM: -239.5022
- Kills / game: 7.8652
- Towers killed: 3.7935
- Towers lost: 9.2294
- NASH%: 37.0381
- DPM: 1749.4049

Role Impact Analysis:
- TOP

In [5]:
print("\n" + "=" * 80)
print("Team Optimization Demo")
print("=" * 80)

print("\nStarting optimization of random team...")
print(f"Initial win rate: {random_win_rate:.4f} ({random_win_rate*100:.2f}%)")

# Run optimization
optimized_team, improvement_log = create_optimal_team(
    random_team, 
    player_to_feature_models, 
    feature_to_win_rate_model, 
    key_features,
    n_iterations=50  # Increase for better results
)

# Plot optimization progress
fig = plot_optimization_progress(improvement_log, title="Team Optimization Progress")
plt.savefig('optimization_progress.png')
plt.close()
print("Optimization progress plot saved to 'optimization_progress.png'")

# Analyze optimized team
optimized_win_rate, optimized_analysis = evaluate_team_composition(
    optimized_team, player_to_feature_models, feature_to_win_rate_model, key_features
)
print("\nAnalyzing optimized team...")
print_team_analysis(optimized_analysis)

# Compare original and optimized teams
print("\nImprovement Summary:")
print(f"Original win rate: {random_analysis['win_rate_percent']:.2f}%")
print(f"Optimized win rate: {optimized_analysis['win_rate_percent']:.2f}%")
print(f"Absolute improvement: {optimized_analysis['win_rate_percent'] - random_analysis['win_rate_percent']:.2f}%")
print(f"Relative improvement: {((optimized_analysis['win_rate'] / random_analysis['win_rate']) - 1) * 100:.2f}%")

# Step 5: Team Variant Comparison
print("\n" + "=" * 80)
print("Team Variant Comparison")
print("=" * 80)

# Create variants of the strong team
print("\nCreating variants of the strong team...")
team_variants = create_team_variants(strong_team, n_variants=3)

# Analyze all variants
variant_analyses = []
for i, variant in enumerate(team_variants):
    print(f"Analyzing Team Variant {i+1}...")
    _, analysis = evaluate_team_composition(
        variant, player_to_feature_models, feature_to_win_rate_model, key_features
    )
    variant_analyses.append(analysis)
    print(f"Win Rate: {analysis['win_rate_percent']:.2f}%")

# Plot comparison
fig = plot_team_comparison(variant_analyses, title="Team Variants Win Rate Comparison")
plt.savefig('team_variants_comparison.png')
plt.close()
print("Team variants comparison plot saved to 'team_variants_comparison.png'")

print("\nDemonstration completed successfully!")
print("You can now use these functions to analyze and optimize your own team compositions.")


Team Optimization Demo

Starting optimization of random team...
Initial win rate: 0.5581 (55.81%)
Iteration 10: Improved win rate to 0.6580
Optimization progress plot saved to 'optimization_progress.png'

Analyzing optimized team...

----- Team Composition Analysis -----
Predicted Win Rate: 66.05%

Predicted Team Stats:
- K:D: 1.2466
- GPM: 1884.5504
- GDM: 156.5889
- Kills / game: 13.9372
- Towers killed: 8.1908
- Towers lost: 4.4869
- NASH%: 77.3314
- DPM: 2354.8950

Role Impact Analysis:
- TOP: Impact Score = 6.07
- JUNGLE: Impact Score = 6.58
- MID: Impact Score = 9.66
- ADC: Impact Score = 8.71
- SUPPORT: Impact Score = 4.59

Team Strengths:
- K:D: 1.25
- GPM: 1884.55
- GDM: 156.59
- Kills / game: 13.94
- DPM: 2354.89

No significant weaknesses detected.

Recommendations:
- This team composition has good potential for success.
- Maintain consistent performance to achieve the predicted win rate.
- The ADC player should focus on positioning to improve survival in fights.

Improveme